In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
load_url = "https://www.f-takken.com/freins/rent/houses/area/items?listtype=houses&lang=ja&location=area&locate%5B0%5D=40131&locate%5B1%5D=40132&locate%5B2%5D=40133&locate%5B3%5D=40134&locate%5B4%5D=40135&locate%5B5%5D=40136&locate%5B6%5D=40137&order1=nn&order2=&limit=20&data_21=0&data_22=0&data_30=0&data_31=0&data_486=&data_486_code=&data_487=&data_487_code=&page=1&currentTabIndex=0"

In [4]:
print(load_url)

https://www.f-takken.com/freins/rent/houses/area/items?listtype=houses&lang=ja&location=area&locate%5B0%5D=40131&locate%5B1%5D=40132&locate%5B2%5D=40133&locate%5B3%5D=40134&locate%5B4%5D=40135&locate%5B5%5D=40136&locate%5B6%5D=40137&order1=nn&order2=&limit=20&data_21=0&data_22=0&data_30=0&data_31=0&data_486=&data_486_code=&data_487=&data_487_code=&page=1&currentTabIndex=0


In [6]:
r = requests.get(load_url)
soup = BeautifulSoup(r.text)